Import thư viện

In [69]:
import pandas as pd
import ast

# Đọc file behaviors.tsv (user, lịch sử đọc, tin được hiển thị)
behaviors = pd.read_csv(r'MINDsmall_dev\behaviors.tsv', sep='\t', header=None,
                        names=['impression_id', 'user_id', 'time', 'history', 'impressions'])

# Đọc file news.tsv (tin tức, thể loại, entity liên quan)
news = pd.read_csv(r'MINDsmall_dev\news.tsv', sep='\t', header=None,
                   names=['news_id', 'category', 'subcategory', 'title', 'abstract', 'url', 'entities', 'abstract_entities'])

print("✅ behaviors:", behaviors.shape)
print("✅ news:", news.shape)
behaviors.head()





✅ behaviors: (73152, 5)
✅ news: (42416, 8)


,impression_id,user_id,time,history,impressions
0,1,U80234,11/15/2019 12:37:50 PM,N55189 N46039 N51741 N53234 N11276 N264 N40716...,N28682-0 N48740-0 N31958-1 N34130-0 N6916-0 N5...
1,2,U60458,11/15/2019 7:11:50 AM,N58715 N32109 N51180 N33438 N54827 N28488 N611...,N20036-0 N23513-1 N32536-0 N46976-0 N35216-0 N...
2,3,U44190,11/15/2019 9:55:12 AM,N56253 N1150 N55189 N16233 N61704 N51706 N5303...,N36779-0 N62365-0 N58098-0 N5472-0 N13408-0 N5...
3,4,U87380,11/15/2019 3:12:46 PM,N63554 N49153 N28678 N23232 N43369 N58518 N444...,N6950-0 N60215-0 N6074-0 N11930-0 N6916-0 N248...
4,5,U9444,11/15/2019 8:25:46 AM,N51692 N18285 N26015 N22679 N55556,N5940-1 N23513-0 N49285-0 N23355-0 N19990-0 N3...


In [62]:
news.head()

,news_id,category,subcategory,title,abstract,url,entities,abstract_entities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N18955,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


Giữ lại các cột cần thiết

In [70]:
behaviors = behaviors[['user_id', 'history', 'impressions']]
news = news[['news_id', 'category', 'subcategory', 'title', 'entities']]


Chuẩn hóa cột entities trong news

In [71]:
def extract_wikidata_ids(entity_str):
    try:
        entities = ast.literal_eval(entity_str)
        return [e['WikidataId'] for e in entities if 'WikidataId' in e]
    except:
        return []

news['entity_ids'] = news['entities'].apply(extract_wikidata_ids)
news.drop(columns=['entities'], inplace=True)

news.head()


,news_id,category,subcategory,title,entity_ids
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","[Q80976, Q43274, Q9682]"
1,N18955,health,medical,Dispose of unwanted prescription drugs during ...,[Q622899]
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,[]
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,[]
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...",[Q3179593]


Tách danh sách history và impressions

In [72]:
# Xử lý lịch sử đọc
behaviors['history'] = behaviors['history'].fillna('').apply(lambda x: x.split(' ') if x != '' else [])
behaviors['impressions'] = behaviors['impressions'].apply(
    lambda x: [i.split('-') for i in x.split(' ')]
)
behaviors.head()


,user_id,history,impressions
0,U80234,"[N55189, N46039, N51741, N53234, N11276, N264,...","[[N28682, 0], [N48740, 0], [N31958, 1], [N3413..."
1,U60458,"[N58715, N32109, N51180, N33438, N54827, N2848...","[[N20036, 0], [N23513, 1], [N32536, 0], [N4697..."
2,U44190,"[N56253, N1150, N55189, N16233, N61704, N51706...","[[N36779, 0], [N62365, 0], [N58098, 0], [N5472..."
3,U87380,"[N63554, N49153, N28678, N23232, N43369, N5851...","[[N6950, 0], [N60215, 0], [N6074, 0], [N11930,..."
4,U9444,"[N51692, N18285, N26015, N22679, N55556]","[[N5940, 1], [N23513, 0], [N49285, 0], [N23355..."


In [ ]:
records = []

for idx, row in behaviors.iterrows():
    user_id = row['user_id']
    history = row['history']
    
    for news_id, clicked in row['impressions']:
        records.append({
            'user_id': user_id,
            'history': history,
            'news_id': news_id,
            'clicked': int(clicked)
        })

# Chuyển sang DataFrame mới
behaviors_expanded = pd.DataFrame(records)

print(behaviors_expanded.head())


  user_id                                            history news_id  clicked
0  U80234  [N55189, N46039, N51741, N53234, N11276, N264,...  N28682        0
1  U80234  [N55189, N46039, N51741, N53234, N11276, N264,...  N48740        0
2  U80234  [N55189, N46039, N51741, N53234, N11276, N264,...  N31958        1
3  U80234  [N55189, N46039, N51741, N53234, N11276, N264,...  N34130        0
4  U80234  [N55189, N46039, N51741, N53234, N11276, N264,...   N6916        0


In [76]:
# Đọc file news.tsv
news = pd.read_csv(
    "MINDsmall_dev/news.tsv",
    sep="\t",
    names=["news_id", "category", "subcategory", "title", "abstract", "url", "title_entities", "abstract_entities"]
)

# Gộp hai cột entity lại để đơn giản hóa
news["entities"] = news["title_entities"].fillna("[]")

# Hàm trích xuất WikidataId
def extract_wikidata_ids(entity_str):
    try:
        entities = ast.literal_eval(entity_str)
        return [e["WikidataId"] for e in entities if "WikidataId" in e]
    except:
        return []

# Tạo cột entity_ids
news["entity_ids"] = news["entities"].apply(extract_wikidata_ids)

# Giữ lại các cột cần thiết
news = news[["news_id", "category", "subcategory", "title", "entity_ids"]]

print(news.head())


  news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N18955     health          medical   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   

                                               title               entity_ids  
0  The Brands Queen Elizabeth, Prince Charles, an...  [Q80976, Q43274, Q9682]  
1  Dispose of unwanted prescription drugs during ...                [Q622899]  
2  The Cost of Trump's Aid Freeze in the Trenches...                       []  
3  I Was An NBA Wife. Here's How It Affected My M...                       []  
4  How to Get Rid of Skin Tags, According to a De...               [Q3179593]  


In [77]:
# Đọc file behaviors.tsv
behaviors = pd.read_csv(
    "MINDsmall_dev/behaviors.tsv",
    sep="\t",
    names=["impression_id", "user_id", "time", "history", "impressions"]
)

# Xử lý history (chuyển chuỗi -> list)
behaviors["history"] = behaviors["history"].fillna("").apply(lambda x: x.split(" ") if x != "" else [])

# Tách impressions: "N12345-1 N23456-0" -> [["N12345", "1"], ["N23456", "0"]]
behaviors["impressions"] = behaviors["impressions"].apply(lambda x: [i.split("-") for i in x.split(" ")])

# Mở rộng ra từng dòng
records = []
for _, row in behaviors.iterrows():
    for news_id, clicked in row["impressions"]:
        records.append({
            "user_id": row["user_id"],
            "history": row["history"],
            "news_id": news_id,
            "clicked": int(clicked)
        })

# Tạo DataFrame mới
behaviors_expanded = pd.DataFrame(records)
print(behaviors_expanded.head())


  user_id                                            history news_id  clicked
0  U80234  [N55189, N46039, N51741, N53234, N11276, N264,...  N28682        0
1  U80234  [N55189, N46039, N51741, N53234, N11276, N264,...  N48740        0
2  U80234  [N55189, N46039, N51741, N53234, N11276, N264,...  N31958        1
3  U80234  [N55189, N46039, N51741, N53234, N11276, N264,...  N34130        0
4  U80234  [N55189, N46039, N51741, N53234, N11276, N264,...   N6916        0


In [ ]:
# Gộp behaviors_expanded với news theo news_id
merged = pd.merge(behaviors_expanded, news, on="news_id", how="left")

# Xem kết quả
print(merged.head())


  user_id                                            history news_id  clicked  \
0  U80234  [N55189, N46039, N51741, N53234, N11276, N264,...  N28682        0   
1  U80234  [N55189, N46039, N51741, N53234, N11276, N264,...  N48740        0   
2  U80234  [N55189, N46039, N51741, N53234, N11276, N264,...  N31958        1   
3  U80234  [N55189, N46039, N51741, N53234, N11276, N264,...  N34130        0   
4  U80234  [N55189, N46039, N51741, N53234, N11276, N264,...   N6916        0   

        category   subcategory  \
0         sports  football_nfl   
1      lifestyle        voices   
2         sports  football_nfl   
3             tv  tv-celebrity   
4  entertainment     celebrity   

                                               title  \
0  Browns apologize to Mason Rudolph, call Myles ...   
1  I've been writing about tiny homes for a year ...   
2  Opinion: Colin Kaepernick is about to get what...   
3  The Kardashians Face Backlash Over 'Insensitiv...   
4  THEN AND NOW: What all yo

In [79]:
merged.to_csv('merged_clean.csv', index=False)
print("✅ Đã lưu file: merged_clean.csv")


✅ Đã lưu file: merged_clean.csv
